In [2]:
from swin_unet.config import get_config
from swin_unet.networks.vision_transformer import SwinUnet as ViT_seg

img_size = 224
num_classes = 9

class ModelConfig:
    class DATA:
        IMG_SIZE=224,
    
    class MODEL:
        class SWIN:
            PATCH_SIZE = 4
            IN_CHANS = 3
            EMBED_DIM = 96
            DEPTHS = [2, 2, 6, 2]
            NUM_HEADS = [3, 6, 12, 24]
            WINDOW_SIZE = 7
            MLP_RATIO = 4.
            QKV_BIAS = True
            QK_SCALE = None
            APE = False
            PATCH_NORM = True
        DROP_RATE =  0.0,
        DROP_PATH_RATE =  0.1
        PRETRAIN_CKPT = 'weights/segmentation/swin_tiny_patch4_window7_224.pth'
    
    class TRAIN:
        USE_CHECKPOINT = False

config = ModelConfig()

net = ViT_seg(config, img_size=img_size, num_classes=num_classes).cuda()
net.load_from(config)

SwinTransformerSys expand initial----depths:[2, 2, 6, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.1;num_classes:9
---final upsample expand_first---
pretrained_path:weights/segmentation/swin_tiny_patch4_window7_224.pth
---start load pretrained modle of swin encoder---


/home/vadim/source/aerial_detection/swin_unet/networks/vision_transformer.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_dict = torch.load(pretrained_path, 

In [40]:
# Imports PIL module
from PIL import Image
import torchvision
import torch

# open method used to open different extension image file
transform_norm = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
        ),
    torchvision.transforms.Resize((224,224))
])

im = transform_norm(Image.open(r"1.png").convert('RGB'))
print(im.T.shape)
x = im.unsqueeze(0)

x.shape
# This method will show image in any image viewer

torch.Size([224, 224, 3])


torch.Size([1, 3, 224, 224])

In [44]:
outputs = net(x.cuda())
outputs = torch.argmax(torch.softmax(outputs, dim=1), dim=1, keepdim=True)
outputs

tensor([[[[6, 1, 3,  ..., 4, 3, 5],
          [2, 1, 7,  ..., 1, 7, 3],
          [5, 1, 3,  ..., 1, 0, 1],
          ...,
          [4, 3, 7,  ..., 1, 6, 3],
          [5, 1, 3,  ..., 1, 1, 1],
          [1, 3, 6,  ..., 2, 6, 4]]]], device='cuda:0')

In [46]:
outputs.shape

torch.Size([1, 1, 224, 224])